# Setup for single factor experiment


Using only Cardiff only and changing seeds with using LIME and SHAP default values

Will only be done on google colab T4 gpus

In [ ]:
#%pip install lime shap hf_xet

In [ ]:
# sanity test

from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer
import random
import torch
from typing import List, Union
from tqdm.notebook import tqdm 


def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True



# Set seed for reproducibility
set_seed(42)

# Constants
CLASS_NAMES = ['NOT-HATE', 'HATE']
BATCH_SIZE = 512  # Adjust based on your GPU memory

# Load models and tokenizers once
pipe_cardiff = pipeline("text-classification", 
                       model="cardiffnlp/twitter-roberta-base-hate-latest", 
                       device=0 if torch.cuda.is_available() else -1,
                       batch_size=BATCH_SIZE)

tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

# Create dataset class for more efficient processing
class HateSpeechDataset:
    def __init__(self, texts):
        self.texts = texts
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]


# Unified prediction function for both models
def batch_predict(texts: Union[List[str], np.ndarray], pipeline_fn):
    """Run predictions in batches for efficiency"""
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
        
    dataset = HateSpeechDataset(texts)
    
    # Process in batches
    all_outputs = []
    for i in range(0, len(dataset), BATCH_SIZE):
        batch_texts = [dataset[j] for j in range(i, min(i + BATCH_SIZE, len(dataset)))]
        outputs = pipeline_fn(batch_texts, top_k=2)
        all_outputs.extend(outputs)
    
    # Convert to numpy array with consistent order
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in all_outputs
    ])


# Model-specific prediction functions
def predict_cardiff(texts):
    return batch_predict(texts, pipe_cardiff)



# Initialize explainers
explainer_lime = LimeTextExplainer(class_names=CLASS_NAMES)
explainer_shap_cardiff = shap.Explainer(predict_cardiff, 
                                       masker=shap.maskers.Text(tokenizer_cardiff))

# Helper function for LIME explanations
def lime_explain(text, predictor, num_features=4):
    exp = explainer_lime.explain_instance(text, 
                                          predictor)
    return ", ".join([f"{word}: {weight:.3f}" for word, weight in exp.as_list()])


# Helper function for SHAP explanations
def get_shap_values(text, explainer, class_idx=1):
    shap_vals = explainer([text])
    return ", ".join([
        f"{feature}: {value:.3f}"
        for feature, value in zip(shap_vals[0].data, shap_vals[0].values[:, class_idx])
        if abs(value) > 0.001
    ])


# Sample data
data = {
    'text': [
        "nawt yall niggers ignoring me",
        "and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew",
        "this is a neutral sentence"
    ]
}
annotated_and_targeted_hatespeech_df = pd.DataFrame(data)


# Process all texts in one batch for efficiency
all_texts = annotated_and_targeted_hatespeech_df["text"].tolist()

# Get predictions in batch
cardiff_preds = pipe_cardiff(all_texts) 

# Process results
results = []
for i, text in enumerate(all_texts):
    try:
        # Get predictions from the batch results
        pred_cardiff = cardiff_preds[i]

        # LIME explanations
        lime_exp_cardiff = lime_explain(text, predict_cardiff)

        # SHAP explanations
        shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)

        # Store results
        results.append({
            "text": text,
            "error": None,
            "cardiff_label": pred_cardiff["label"],
            "cardiff_score": pred_cardiff["score"],
            "lime_cardiff": lime_exp_cardiff,
            "shap_cardiff": shap_exp_cardiff
        })
    except Exception as e:
        results.append({
            "text": text,
            "error": f"Caught an exception: {e}",
            "cardiff_label": None,
            "cardiff_score": None,
            "lime_cardiff": None,
            "shap_cardiff": None
        })

final_df = pd.DataFrame(results)


# Print the results
final_df.head()


Device set to use cpu


  0%|          | 0/498 [00:00<?, ?it/s]

,text,error,cardiff_label,cardiff_score,lime_cardiff,shap_cardiff
0,nawt yall niggers ignoring me,None,HATE,0.935527,"niggers: -0.907, nawt: -0.102, me: 0.074, yall...","n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ..."
1,and this is why i end up with nigger trainee d...,None,HATE,0.993975,"nigger: -0.981, with: -0.007, i: 0.004, doctor...","and : 0.003, this : 0.001, n: -0.007, igger : ..."
2,this is a neutral sentence,None,NOT-HATE,0.999157,"a: 0.001, is: 0.001, this: 0.001, sentence: 0....",


In [18]:
# Saving to drive
from google.colab import drive
drive.mount('/content/drive')

# Save to your Google Drive
final_df.to_csv('/content/drive/MyDrive/hate_speech_analysis_results.csv', index=False)
final_df.to_excel('/content/drive/MyDrive/hate_speech_analysis_results.xlsx', index=False)
final_df.to_pickle('/content/drive/MyDrive/hate_speech_analysis_results.pkl')

ModuleNotFoundError: No module named 'google.colab'

In [24]:
import pandas as pd
import requests
from io import BytesIO

# GitHub raw content URL for the pickle file
github_raw_url = "https://github.com/Takosaga/master_thesis/raw/main/data/processed/annotated_and_targeted_hatespeech.pkl" # Updated URL
# Download the pickle file
response = requests.get(github_raw_url)

# Check if the request was successful
if response.status_code == 200:
    # Load the pickle data into a pandas DataFrame
    df = pd.read_pickle(BytesIO(response.content))
    print(f"DataFrame loaded successfully with {len(df)} rows")
    
    # Display the first few rows
    print("\nPreview of the DataFrame:")
    display(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    print(f"Response: {response.text}")

DataFrame loaded successfully with 65217 rows

Preview of the DataFrame:


,id_new,text,extracted_target,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech,platform,original_dataset_title,original_id,original_label,original_target
0,1,i dont think im getting my baby them white 9 h...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179055004553900032,normal,[None]
1,2,we cannot continue calling ourselves feminists...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179063826874032128,normal,[None]
2,3,nawt yall niggers ignoring me,{race},not_hatespeech,normal,normal,twitter,HateXplain,1178793830532956161,normal,[African]
3,4,<user> i am bit confused coz chinese ppl can n...,{race},hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179088797964763136,hatespeech,[Asian]
4,5,this bitch in whataburger eating a burger with...,"{race, gender}",hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179085312976445440,hatespeech,"[Caucasian, Women]"


In [22]:
df[:100].shape

NameError: name 'df' is not defined

In [19]:
def process_dataframe(df, text_column="text"):
    """Process an entire dataframe with efficient batching"""
    all_texts = df[text_column].tolist()
    total_texts = len(all_texts)
    
    # Get predictions in batch
    cardiff_preds = pipe_cardiff(all_texts)
    
    results = []
    for i, text in tqdm(enumerate(all_texts), total=total_texts, desc="Processing texts"):
        try:

            pred_cardiff = cardiff_preds[i]
            
            # Calculate explanations for all examples
            lime_exp_cardiff = lime_explain(text, predict_cardiff)
            shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)
            
            results.append({
                "text": text,
                "error": None,
                "cardiff_label": pred_cardiff["label"],
                "cardiff_score": pred_cardiff["score"],
                "lime_cardiff": lime_exp_cardiff,
                "shap_cardiff": shap_exp_cardiff,
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": f"Caught an exception: {e}",
                "cardiff_label": None,
                "cardiff_score": None,
                "lime_cardiff": None,
                "shap_cardiff": None,
            })
    
    return pd.DataFrame(results)

In [ ]:
section = len(df)//1000

In [ ]:
def save_dataframe(df, name, base_path='/content/drive/MyDrive/'):
    """
    Save a dataframe to CSV, Excel, and pickle formats.
    
    Parameters:
    - df: pandas DataFrame to save
    - name: base name for the files (without extension)
    - base_path: directory path where files will be saved
    """
    # Save as CSV
    df.to_csv(f'{base_path}{name}.csv', index=False)
    
    # Save as Excel
    df.to_excel(f'{base_path}{name}.xlsx', index=False)
    
    # Save as pickle
    df.to_pickle(f'{base_path}{name}.pkl')

In [ ]:
set_seed(42)
df_1 = process_dataframe(df[:section])
save_dataframe(df_1, 'df_1_seed42')

In [ ]:
set_seed(0)
df_1 = process_dataframe(df[:section])
save_dataframe(df_1, 'df_1_seed0')